In [11]:
from tensorflow.keras.layers import Input, Embedding, GRU, dot, Dense, add, concatenate, multiply, Subtract
from tensorflow import keras
import numpy as np
import tensorflow.keras.backend as K

In [3]:
class rnn_neg_sampling(keras.models.Model):
    def __init__(self, vocab_size, dim, seq_length, num_negatives=10):
        super(rnn_neg_sampling, self).__init__()
        
        self.num_negatives = num_negatives 
        
        self.embedding_input = Embedding(
            input_dim=vocab_size+1, 
            output_dim=dim,
            mask_zero=True,
            input_length=seq_length
        )
        self.embedding_output = Embedding(
            input_dim=vocab_size+1, 
            output_dim=dim,
            mask_zero=True,
            input_length=seq_length
        )
        
        self.gru = GRU(dim, return_sequences=True)
        
    def call(self, input):
        inputs, targets, negatives = input
        
        embedded_inputs = self.embedding_input(inputs)
        # https://keras.io/layers/recurrent/#gru
        gru_output = self.gru(embedded_inputs)

        embedded_targets = self.embedding_output(targets)
        embedded_negatives = self.embedding_output(negatives)

        positive_dots = K.expand_dims(multiply([gru_output, embedded_targets]), axis=1)
        negative_dots = multiply([K.repeat_elements(K.expand_dims(gru_output, axis=1),
                                                      rep=self.num_negatives, 
                                                      axis=1),
                                    embedded_negatives
                                   ])
        
        positive_logits = K.sum(positive_dots, axis=-1)
        negative_logits = -K.sum(negative_dots, axis=-1)
        logits = K.concatenate([positive_logits, negative_logits], axis=1)
        
        self.add_loss(-K.mean(K.log(K.sigmoid(logits))))

In [17]:
X_inputs = np.random.randint(0, 5, size=(batch_size, sequence_length))
X_targets = np.random.randint(0, 5, size=(batch_size, sequence_length))
X_negatives = np.random.randint(0, 5, size=(batch_size, neg_number, sequence_length))

neg_number = 3
vector_dim=10
sequence_length = 5
vocab_size = 100
batch_size = 3

In [18]:
ns_model = rnn_neg_sampling(vocab_size, vector_dim, sequence_length, neg_number)

In [19]:
ns_model.compile(loss=None, optimizer='rmsprop')
history = ns_model.fit([X_inputs, X_targets, X_negatives], epochs=1)

AttributeError: 'rnn_neg_sampling' object has no attribute 'total_loss'